In [2]:
import h5py

In [35]:
# hdf5_file = "/local-fs/bachelor-students/2021-2022/Thema12/ra_data/stripes4/dataset/train.h5py"
# f = h5py.File(hdf5_file, 'r')
# # x_train = f.get('x_train')
# # y_train = f.get('y_train')
# x_val = f.get('x_test')
# y_val = f.get('y_test')

In [1]:
import PIL
from PIL import Image
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import notebook_config as cfg

In [3]:
img = cv.imread(f"/commons/Themas/Thema11/Giepmans/work/tmp/full_white.tif", cv.IMREAD_GRAYSCALE)

In [4]:
print(np.unique(img))

[255]


(5, 6)
(48, 8)
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
(64, 8, 64, 8)


In [23]:
def window(array, window_size=(3,3)):
    x_size = array.shape[0]
    y_size = array.shape[1]
    
    horizontal_pad = window_size[0] // 2
    vertical_pad = window_size[1] // 2
    array = np.pad(array, [(horizontal_pad,vertical_pad), (horizontal_pad,vertical_pad)],
               mode = 'constant', constant_values = 0)
    print(array)
    print(array.strides)
    strides = array.strides * 2
    print(strides)
    
    #x_size = (array.shape[0] - window_size[0]) + 1
    #y_size = (array.shape[1] - window_size[1]) + 1
#     x_size = array.shape[0]
#     y_size = array.shape[1]
    shape = (x_size, y_size) + window_size
    return np.lib.stride_tricks.as_strided(array, shape, strides)

In [24]:
print(img.shape)
print(img.strides)
windows = window(img)

(326, 652)
(652, 1)
[[  0   0   0 ...   0   0   0]
 [  0 255 255 ... 255 255   0]
 [  0 255 255 ... 255 255   0]
 ...
 [  0 255 255 ... 255 255   0]
 [  0 255 255 ... 255 255   0]
 [  0   0   0 ...   0   0   0]]
(654, 1)
(654, 1, 654, 1)


In [32]:
a = np.array([[ 1.,  1.,  1., 1., 1., 1.],
              [ 1.,  1.,  1., 1., 1., 1.],
              [ 1.,  1.,  1., 1., 1., 1.],
              [ 1.,  1.,  1., 1., 1., 1.],
              [ 1.,  1.,  1., 1., 1., 1.],
             [ 1.,  1.,  1., 1., 1., 1.]])
print(a.shape)
print(a.strides)


windows = window(a, (5,5))

(6, 6)
(48, 8)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
(80, 8)
(80, 8, 80, 8)


In [26]:
def windows_to_dataset(windows):
    array = None
    
    for i in range(windows.shape[0]):
        for b in range(windows.shape[1]):
            if array is None:
                array = windows[i, b,:,:].flatten()
                array = np.expand_dims(array, axis=0)
                #print(array)
                #print(array.shape)
            else:
                #print( windows[i, b,:,:].flatten())
                array = np.append(array, np.expand_dims(windows[i, b,:,:].flatten(), axis=0), axis=0)
            #print(windows[i, b,:,:])
            #print(windows[i, b,:,:].flatten())
    return array

In [33]:
arr = windows_to_dataset(windows)
print(arr.shape)

print(windows.shape)
axis0 = windows.shape[0] * windows.shape[1]
axis1= windows.shape[2] * windows.shape[3]
arr2 = windows.reshape(axis0, axis1)
print(arr2)

(36, 25)
(6, 6, 5, 5)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1.
  1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1.
  1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1.
  1.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1.
  1.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1.
  0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0.
  0.]
 [0. 0. 1. 1. 1. 0

In [34]:
print(arr2[-1])

[1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
